# Comparador de Constitucion Chile - España

Usaremos LangChain para comparar la constitución actual chilena (del 80') con la española.

In [35]:
from functools import partial

from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader

## Setup

Cargamos los documentos en PDF directamente

In [42]:
from langchain.document_loaders import UnstructuredBaseLoader

ImportError: cannot import name 'UnstructuredBaseLoader' from 'langchain.document_loaders' (/Users/tcapelle/miniforge3/envs/openai/lib/python3.10/site-packages/langchain/document_loaders/__init__.py)

In [41]:
UnstructuredBaseLoader??

Object `UnstructuredBaseLoader` not found.


In [36]:
pdf_loader = partial(UnstructuredPDFLoader, remove_newline=True)

In [37]:
dl = DirectoryLoader(path="./data", 
                     glob="Constitucion*", 
                     loader_cls=pdf_loader)

In [38]:
docs = dl.load()

TypeError: partition_pdf() got an unexpected keyword argument 'remove_newline'

Chile

In [25]:
docs[0].page_content[:400]

'CONSTITUCIÓN POLÍTICA DE LA REPÚBLICA\n\nTexto actualizado a octubre  de 2010\n\n2\n\n3\n\nCONSTITUCIÓN POLÍTICA DE LA REPÚBLICA DE CHILE\n\nINDICE\n\nCAPÍTULOS\n\nCAPITULO I  BASES DE LA INSTITUCIONALIDAD\n\nCAPITULO II  NACIONALIDAD Y CIUDADANIA\n\nCAPITULO III  DE LOS DERECHOS Y DEBERES\n\nCONSTITUCIONALES\n\nCAPITULO IV  GOBIERNO\n\nPresidente de la República\n\nMinistros de Estado\n\nArts.   1 al 9\n\nArts.  10 al 18\n\nArt'

España

In [14]:
docs[1].page_content[:400]

'CONSTITUCIÓN\n\nESPAÑOLA\n\nAprobada por Las Cortes en sesiones plenarias del\n\nCongreso de los Diputados y del Senado celebradas\n\nel 31 de octubre de 1978\n\nRatificada por el pueblo español en referéndum de\n\n6 de diciembre de 1978\n\nSancionada por S. M. el Rey ante Las Cortes\n\nel 27 de diciembre de 1978\n\n© Agencia Estatal Boletín Oficial del Estado\n\nNIPO: 007-14-093-3\n\nCatálogo de Publicaciones de la Ad'

Hacemos split en pedazos para mandarlo al embedding...

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
splitted_docs = text_splitter.split_documents(docs)

let's embed using OpenAI Ada02

In [19]:
embeddings = OpenAIEmbeddings(chunk_size=1024)
# db = Chroma.from_documents(texts, embeddings)
db = FAISS.from_documents(splitted_docs, embeddings)

We store everything in the vector database `db`

In [8]:
retriever = db.as_retriever()

Creamos un chat para recuperar info de la base de datos...

In [23]:
ConversationalRetrievalChain??

Init signature:
ConversationalRetrievalChain(
    *,
    memory: Optional[langchain.schema.BaseMemory] = None,
    callback_manager: langchain.callbacks.base.BaseCallbackManager = None,
    verbose: bool = None,
    combine_docs_chain: langchain.chains.combine_documents.base.BaseCombineDocumentsChain,
    question_generator: langchain.chains.llm.LLMChain,
    output_key: str = 'answer',
    return_source_documents: bool = False,
    get_chat_history: Optional[Callable[[Tuple[str, str]], str]] = None,
    retriever: langchain.schema.BaseRetriever,
    max_tokens_limit: Optional[int] = None,
) -> None
Source:        
class ConversationalRetrievalChain(BaseConversationalRetrievalChain, BaseModel):
    """Chain for chatting with an index."""

    retriever: BaseRetriever
    """Index to connect to."""
    max_tokens_limit: Optional[int] = None
    """If set, restricts the docs to return from store based on tokens, enforced only
    for StuffDocumentChain"""

    def _reduce_tokens_below_li

In [11]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(), 
                                           retriever, 
                                           return_source_documents=True)

## Comparemos las constituciones...

In [12]:
chat_history = []
query = "Cual pais tiene un sistema parlamentario?"
result = qa({"question": query, "chat_history": chat_history})

In [13]:
result['answer']

'Varios países utilizan un sistema parlamentario, como Reino Unido, Canadá, Australia, Nueva Zelanda, Japón, India, entre otros. ¿Quieres saber sobre un país en particular?'

In [14]:
result['source_documents'][0]

Document(page_content='La Cámara de Diputados se renovará en su totalidad cada cuatro años.114\n\nArtículo 48. Para ser elegido diputado  se requiere ser ciudadano con derecho a\n\nsufragio,  tener  cumplidos  veintiún  años  de  edad,  haber  cursado  la  Enseñanza  Media  o\n\nequivalente  y  tener  residencia  en  la  región  a  que  pertenezca  el  distrito  electoral\n\ncorrespondiente durante un plazo no inferior a dos años, contando hacia atrás desde el\n\ndía de la elección.115\n\nArtículo 49. El Senado se compone de miembros elegidos en votación directa por\n\ncircunscripciones senatoriales, en consideración a las regiones del país, cada una de las\n\n111 Artículo sustituido por el artículo 1º, Nº 20 de la ley de Reforma Constitucional Nº 20.050.\n\n112 Véase la  ley Orgánica Constitucional del Congreso Nacional Nº 18.918.\n\n113 Véase el Título Final de la ley Nº 18.700 Orgánica Constitucional sobre Votaciones  Populares y Escrutinios.', metadata={'source': 'data/Constitucion

In [15]:
chat_history = [(query, result["answer"])]
query = "Y chile?"
result = qa({"question": query, "chat_history": chat_history})

In [16]:
result['answer']

'Chile tiene un sistema presidencialista, en el cual el Presidente de la República es el jefe de Estado y de gobierno. Si bien el país cuenta con un Congreso Nacional, este no tiene un rol predominante en la definición y conducción de la política gubernamental.'

In [17]:
result['source_documents'][0]

Document(page_content='2 Inciso modificado por el artículo único, N° 1 de la ley de Reforma Constitucional N° 19.611.\n\nArtículo\n\n2°\n\nSon\n\nemblemas  nacionales  la  bandera  nacional,  el  escudo\n\nde armas de la República y el himno nacional.\n\n6\n\nArtículo 3º El Estado de Chile es unitario.\n\nLa  administración  del  Estado será  funcional  y  territorialmente descentralizada, o\n\ndesconcentrada en su caso, de conformidad a la ley.\n\nLos  órganos  del  Estado  promoverán  el  fortalecimiento  de  la  regionalización  del\n\npaís  y  el  desarrollo  equitativo  y  solidario  entre  las  regiones,  provincias  y  comunas  del\n\nterritorio nacional.3\n\nArtículo 4° Chile es una república democrática.\n\nArtículo  5°  La  soberanía  reside  esencialmente  en  la  Nación.  Su  ejercicio  se\n\nrealiza por el pueblo a través del plebiscito y de elecciones periódicas y, también, por las\n\nautoridades que esta Constitución establece. Ningún sector del pueblo ni individuo algun

In [18]:
chat_history = [(query, result["answer"])]
query = "Puedes explicarme la diferencia entre los dos?"
result = qa({"question": query, "chat_history": chat_history})

In [19]:
result['answer']

'En un sistema presidencialista, el presidente es el jefe de estado y de gobierno y tiene poderes ejecutivos y legislativos para tomar decisiones sin tener que depender del Congreso para aprobar medidas. Mientras que en un sistema en el que el Congreso tiene un rol predominante, el poder ejecutivo es ejercido por un primer ministro o un jefe de gobierno y depende de la aprobación del Congreso para llevar a cabo sus medidas. En este sistema, el Congreso tiene un papel activo en la definición y conducción de la política gubernamental y el poder del jefe de gobierno está más limitado que en un sistema presidencialista.'

In [20]:
chat_history = [(query, result["answer"])]
query = "En españa el rey tiene algun poder?"
result = qa({"question": query, "chat_history": chat_history})

In [21]:
result['answer']

'En España, el rey tiene algunos poderes ejecutivos, como el derecho de sancionar y promulgar las leyes, convocar y disolver las Cortes Generales y nombrar y separar a los miembros del Gobierno a propuesta de su Presidente, entre otros. También tiene algunos poderes simbólicos, como prestar juramento de fidelidad al país y recibir acreditaciones de embajadores extranjeros.\n\nComparado con sistemas presidencialistas, como en Estados Unidos, el presidente tiene un poder ejecutivo mucho mayor y su elección es directa. En el caso de España, el Rey tiene un poder simbólico muy importante y no es elegido directamente, sino que su sucesión se define por la ley. \n\nEn sistemas parlamentarios donde el Congreso tiene un rol predominante, como en el Reino Unido, el monarca tiene aún menos poderes ejecutivos y su principal rol es representativo, como en España. En ambos sistemas, el líder del gobierno es elegido por el parlamento o por los partidos políticos, aunque en España esto se hace a part

In [22]:
result["source_documents"][0]

Document(page_content='el nombramiento del Presidente del Gobierno, y la disolución prevista\n\nen el artículo 99, serán refrendados por el Presidente del Congreso.\n\n2.    De los actos del Rey serán responsables las personas que los\n\n22\n\nrefrenden.\n\nArtículo 65.\n\n1.    El Rey recibe de los Presupuestos del Estado una cantidad glo-\n\nbal para el sostenimiento de su Familia y Casa, y distribuye libremen-\n\nte la misma.\n\n2.    El Rey nombra y releva libremente a los miembros civiles y\n\nmilitares de su Casa.\n\nLa Casa del\n\nRey\n\nTÍTULO III\n\nDe las Cortes Generales\n\nCAPÍTULO PRIMERO\n\nDe las Cámaras\n\nArtículo 66.\n\n1.    Las Cortes Generales representan al pueblo español y están\n\nformadas por el Congreso de los Diputados y el Senado.\n\n2.    Las Cortes Generales ejercen la potestad legislativa del Esta-\n\ndo, aprueban sus Presupuestos, controlan la acción del Gobierno y\n\ntienen las demás competencias que les atribuya la Constitución.\n\n3.    Las Cortes Gen